In [4]:
import ee, os
import geemap
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:1082'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:1082'
Map = geemap.Map(center=(32.4, 116.7), zoom=5)

In [5]:
year = 2021
start_date = str(year) + '-01-01'
end_date = str(year) + '-12-31'
print(start_date, end_date)
#region = geemap.shp_to_ee('D:/Fudan/China_boundary/Anhui.shp')
region = ee.FeatureCollection('users/wangxinxin0803/Provinces/Anhui')
# ee.Initialize()
Map.addLayer(region)
Map

2021-01-01 2021-12-31


Map(center=[32.4, 116.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [6]:
######cloud masking for Landsat
def maskL457sr(image):
      # Bit 0 - Fill
      # Bit 1 - Dilated Cloud
      # Bit 2 - Unused
      # Bit 3 - Cloud
      # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBand, None, True)\
        .updateMask(qaMask)\
        .updateMask(saturationMask)

def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBand, None, True) \
        .updateMask(qaMask)\
        .updateMask(saturationMask)

In [7]:
# -------------------------------------------------------
# 3  ImageCollection 
# -------------------------------------------------------
# function for EVI,GCVI,NDVI,NDWI,LSWI
def ND_VI(img, b1, b2, bName):
    VI = img.normalizedDifference([b1,b2]).rename(bName)
    return VI.updateMask(VI.gt(-1).And(VI.lt(1)))

def f_EVI(img,b1,b2,b3):
    EVI = img.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
          'NIR': img.select(b3),
          'RED': img.select(b2),
          'BLUE':img.select(b1)}).rename('EVI')
    return EVI.updateMask(EVI.gt(-1)).And(EVI.lt(1))

def addVIs(img):
    NDVI = ND_VI(img,'B4','B3','NDVI')
    mNDWI = ND_VI(img,'B2','B5','mNDWI')
    LSWI = ND_VI(img,'B4','B5','LSWI')
    EVI = f_EVI(img,'B1','B3','B4')
    return img.addBands(NDVI).addBands(LSWI).addBands(EVI).addBands(mNDWI)

In [8]:
#water detection
def Water(img):
    return img.select('mNDWI').gt(img.select('EVI'))\
        .Or(img.select('mNDWI').gt(img.select('NDVI')))\
        .And(img.select('EVI').lt(0.1))

#Non-Water detection
def NonWater(image):
    return image.select('mNDWI').lte(image.select('EVI'))\
        .And(image.select('mNDWI').lte(image.select('NDVI')))\
        .Or(image.select('EVI').gte(0.1))

In [9]:
collection5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')\
    .filterBounds(region)\
    .filterDate(start_date, end_date)\
    .map(maskL457sr)\
    .select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])\
    .map(addVIs)
print('Landsat 5')
print(collection5.size().getInfo())

collection7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')\
    .filterBounds(region)\
    .filterDate(start_date, end_date)\
    .map(maskL457sr)\
    .select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])\
    .map(addVIs)
print('Landsat 7')
print(collection7.size().getInfo())

# get landsat 8
collection8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
    .filterBounds(region)\
    .filterDate(start_date, end_date)\
    .map(maskL8sr)\
    .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])\
    .map(addVIs)
print('Landsat 8')
print(collection8.size().getInfo())

# get landsat 9
collection9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')\
    .filterBounds(region)\
    .filterDate(start_date, end_date)\
    .map(maskL8sr)\
    .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])\
    .map(addVIs)
print('Landsat 9')
print(collection9.size().getInfo())

Landsat 5
0
Landsat 7
296
Landsat 8
285
Landsat 9
51


In [10]:
#Merge to a single collection
col1 = ee.ImageCollection(collection5.merge(collection7))
collection1 = ee.ImageCollection(col1.merge(collection8))
collection = ee.ImageCollection(collection1.merge(collection9))
print(collection.size().getInfo())

632


In [11]:
#water frequency and quality band (number of good observations)
total_Pos = collection.map(Water).sum()
total_Neg = collection.map(NonWater).sum()

In [12]:
#water frequency 
flooding_freq_export = (total_Pos.divide(total_Pos.add(total_Neg))).multiply(254).add(ee.Image(1)).rename('flooding_freq_export')
goodGuals = total_Pos.add(total_Neg).rename('goodGuals');
flooding_freq = (total_Pos.divide(total_Pos.add(total_Neg))).rename('flooding_freq')

#year-long surface water
per_water = ee.Image(1).updateMask(flooding_freq.gte(0.75)).clip(region).rename('per_water')

#Seasonal surface water: water frequency >= 0.05 and water frequency < 0.75 
sea_water = ee.Image(1).updateMask(flooding_freq.gte(0.05).And(flooding_freq.lt(0.75))).clip(region).rename('sea_water')

#Ephemeral surface water: water frequency < 0.05
eph_water = ee.Image(1).updateMask(flooding_freq.lt(0.05)).clip(region).rename('eph_water')

Map.addLayer(flooding_freq_export.clip(region),{},'flooding_freq_export',True);
Map.addLayer(per_water.clip(region),{}, 'per_water',True);
#Map.addLayer(sea_water.clip(region),{}, 'sea_water',false);
#Map.addLayer(eph_water.clip(region),{}, 'eph_water',false);
Map

Map(bottom=3616.0, center=[32.4, 116.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…